# This bot has been created by Robert Kaczmarek(38657)                                                                             & Michał Żelazo(38685). 
It has been based on "Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems" 1st Edition by Aurélien Géron.

### Install process

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.


### Libraries

In [3]:
from newspaper import Article
import random
import string 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings
#Just to ignore any warning messages
warnings.filterwarnings('ignore')

In [4]:
nltk.download('punkt', quiet=True)

True

In [5]:
nltk.download('wordnet', quiet=True)

True

### Article's URL

In [7]:
article = Article('https://en.wikipedia.org/wiki/Coronavirus_disease_2019')
article.download() 
article.parse()
article.nlp()
article1 = Article('https://www.mayoclinic.org/diseases-conditions/coronavirus/diagnosis-treatment/drc-20479976')
article1.download() 
article1.parse()
article1.nlp()
article2 = Article('https://www.mayoclinic.org/diseases-conditions/coronavirus/symptoms-causes/syc-20479963')
article2.download() 
article2.parse()
article2.nlp()    
article3 = Article('https://www.nytimes.com/article/prepare-for-coronavirus.html')
article3.download() 
article3.parse()
article3.nlp()
corpus = article.text+article1.text+article2.text+article3.text

In [9]:
print(corpus)

Viral respiratory disease first detected in 2019

Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).[8] The disease was first identified in December of 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.[9][10] Common symptoms include fever, cough, and shortness of breath.[5] Other symptoms may include fatigue, muscle pain, diarrhea, sore throat, loss of smell, and abdominal pain.[5][11][12] The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days.[5][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.[9][14] As of 14 April 2020, more than 1.93 million[7] cases have been reported across 210 countries and territories,[15] resulting in over 120,000 deaths. More than 464,000 people have recovered.[7]

T

### Tokenizine the data

In [10]:
text = corpus
sent_tokens = nltk.sent_tokenize(text)

In [11]:
print(sent_tokens)

['Viral respiratory disease first detected in 2019\n\nCoronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).', "[8] The disease was first identified in December of 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.", '[9][10] Common symptoms include fever, cough, and shortness of breath.', '[5] Other symptoms may include fatigue, muscle pain, diarrhea, sore throat, loss of smell, and abdominal pain.', '[5][11][12] The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days.', '[5][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.', '[9][14] As of 14 April 2020, more than 1.93 million[7] cases have been reported across 210 countries and territories,[15] resulting in over 120,000 deaths.', 'More than 464,0

## A Lemmatization Function

In [12]:
remove_punct_dict = dict(  (ord(punct), None) for punct in string.punctuation)

In [13]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [14]:
print(remove_punct_dict)

{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [15]:
def LemNormalize(text):
    return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

In [16]:
print(LemNormalize(text))

['viral', 'respiratory', 'disease', 'first', 'detected', 'in', '2019', 'coronavirus', 'disease', '2019', 'covid19', 'is', 'an', 'infectious', 'disease', 'caused', 'by', 'severe', 'acute', 'respiratory', 'syndrome', 'coronavirus', '2', 'sarscov28', 'the', 'disease', 'was', 'first', 'identified', 'in', 'december', 'of', '2019', 'in', 'wuhan', 'the', 'capital', 'of', 'chinas', 'hubei', 'province', 'and', 'has', 'since', 'spread', 'globally', 'resulting', 'in', 'the', 'ongoing', '2019–20', 'coronavirus', 'pandemic910', 'common', 'symptoms', 'include', 'fever', 'cough', 'and', 'shortness', 'of', 'breath5', 'other', 'symptoms', 'may', 'include', 'fatigue', 'muscle', 'pain', 'diarrhea', 'sore', 'throat', 'loss', 'of', 'smell', 'and', 'abdominal', 'pain51112', 'the', 'time', 'from', 'exposure', 'to', 'onset', 'of', 'symptoms', 'is', 'typically', 'around', 'five', 'days', 'but', 'may', 'range', 'from', 'two', 'to', 'fourteen', 'days513', 'while', 'the', 'majority', 'of', 'cases', 'result', 'in'

### Key word matching

In [17]:
GREETING_INPUTS = ["hi", "hello","greetings",  "wassup","hey"] 

In [18]:
GREETING_RESPONSES = ["howdy","hi", "hey", "hello" ,"hey there"]

In [19]:
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

### Generating The Chat Bot Response

In [20]:
def response(user_response):
    bot_response='' 
    sent_tokens.append(user_response) 
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    score = flat[-2]
    if(score==0):
        bot_response=bot_response+"I apologize, I don't understand."
    else:
        bot_response = bot_response+sent_tokens[idx]
    sent_tokens.remove(user_response)
    return bot_response

# CoronaBot

In [ ]:
flag=True
print("CoronaBot: I am CoronaBOT or CorBot for short.\nI will answer your queries about Coronavirus(SARS-CoV-2) sings, or coping and supprot.\nI will also tell you when to see a doctor. Just let me know! \nIf you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("CorBot: You're welcome !")
        else:
            if(greeting(user_response)!=None):
                print("CorBot: "+greeting(user_response))
            else:
                print("CorBot: "+response(user_response))
    else:
        flag=False
        print("CorBot: Chat with you later !")

CoronaBot: I am CoronaBOT or CorBot for short.
I will answer your queries about Coronavirus(SARS-CoV-2) sings, or coping and supprot.
I will also tell you when to see a doctor. Just let me know! 
If you want to exit, type Bye!
Hi
CorBot: hello
I have a fever
CorBot: [9][10] Common symptoms include fever, cough, and shortness of breath.
Should i call my doctor?
CorBot: When to see a doctor

If you have COVID-19 symptoms or you've been in contact with someone diagnosed with COVID-19 , contact your doctor or clinic right away for medical advice.
What should I do at home?
CorBot: Your doctor will likely recommend that you stay home for a period of time except to get medical care.
What should i do?
CorBot: I apologize, I don't understand.
When should i see my doctor?
CorBot: When to see a doctor

If you have COVID-19 symptoms or you've been in contact with someone diagnosed with COVID-19 , contact your doctor or clinic right away for medical advice.


In [ ]:
#The bot sometimes demands more talk and then it gets smarter. And also we haven't found any more related URL's with diagnosing. 